In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import preparing_data as F

In [2]:
data=pd.read_pickle("balanced_df_20210908_205836_corr_ONLY_RISKY_EVENTS.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

(196, 381)


,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,OBJECT1_CR_R,...,OBJECT2_CORR_CTDOT_R_4,OBJECT2_CORR_CTDOT_T_4,OBJECT2_CORR_CTDOT_N_4,OBJECT2_CORR_CTDOT_RDOT_4,OBJECT2_CORR_CNDOT_R_4,OBJECT2_CORR_CNDOT_T_4,OBJECT2_CORR_CNDOT_N_4,OBJECT2_CORR_CNDOT_RDOT_4,OBJECT2_CORR_CNDOT_TDOT_4,COLLISSION_PROBABILITY_TARGET
0,34182.0,8743.0,-324.0,27762.0,-19940.6,-38.9,-5101.5,-7100.9,-5.628932,14.571513,...,-0.994164,-0.036543,-0.602419,0.019854,0.580031,0.048458,0.998851,-0.043350,-0.589000,-30.0
1,2227.0,11172.0,101.7,-1463.6,1676.3,-201.4,-8424.3,-7336.0,-4.009306,63.680258,...,-0.965613,0.515422,-0.135461,-0.477611,-0.227886,0.075571,0.510505,-0.068810,0.250754,-30.0
2,21580.0,7105.0,524.9,-19398.7,9440.2,-325.8,-3112.6,-6379.0,-5.063185,16.464136,...,-0.963557,0.001336,0.130138,0.193827,0.162878,-0.622041,-0.964628,0.630162,0.006580,-30.0
3,2851.0,5277.0,150.9,2661.9,-1011.4,16.2,-1873.6,-4933.5,-4.920819,9.112098,...,-0.999617,0.232796,0.082441,-0.237460,0.021630,-0.148656,0.796492,0.154916,-0.018924,-30.0
4,4517.0,6491.0,167.5,-4075.1,1943.6,-55.5,-2794.8,-5859.0,-5.048808,8.987033,...,-0.988245,0.084214,-0.736068,-0.128995,-0.024278,0.058996,0.657088,-0.306377,-0.009793,-30.0


In [3]:
#data=data.head(100)

In [4]:
data.shape

(196, 381)

In [5]:
############################### FEATURE ENGINEERING ##########################################
# Gradient: Miss distance two last CDM
data["_GRADIENT_MISS_DISTANCE_34"]=(-data.MISS_DISTANCE_3+data.MISS_DISTANCE_4)/abs(data.__time_to_tca_4-data.__time_to_tca_3)
# Gradient: Miss distance first and last CDM
data["_GRADIENT_MISS_DISTANCE_14"]=(-data.MISS_DISTANCE+data.MISS_DISTANCE_4)/abs(data.__time_to_tca_4-data.__time_to_tca)
#Gradient: COLLISSION PROBABILITY two last CDM
data["_GRADIENT_PC_34"]=(-data.COLLISSION_PROBABILITY_3+data.COLLISSION_PROBABILITY_4)/abs(data.__time_to_tca_4-data.__time_to_tca_3)
#Gradient: COLLISSION PROBABILITY first and last CDM
data["_GRADIENT_PC_14"]=(-data.COLLISSION_PROBABILITY+data.COLLISSION_PROBABILITY_4)/abs(data.__time_to_tca_4-data.__time_to_tca)

In [6]:
data.shape

(196, 385)

In [7]:
train, test = train_test_split(data, test_size=0.30, random_state=42)

In [8]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 137 x 385
Test dataframe dimension 59 x 385


In [9]:
Y_train = train["COLLISSION_PROBABILITY_TARGET"]
X_train= train.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)
Y_test = test["COLLISSION_PROBABILITY_TARGET"]
X_test= test.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)


In [10]:
X = X_train
y = Y_train

In [11]:
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
      dtrain = lgb.Dataset(data=X, label=y)
      #Metric evaluation functions
      def lgb_r2_score(preds, dtrain):                #R2
            labels = dtrain.get_label()
            return 'metric', r2_score(labels, preds), True
      def lgb_root_squared_error(preds, dtrain):      #RMSE
            labels = dtrain.get_label()
            return 'metric', mean_squared_error(labels, preds,squared=False), True
      def lgb_mean_absolute_error(preds, dtrain):     #MAE
            labels = dtrain.get_label()
            return 'metric', mean_absolute_error(labels, preds), True
      def lgb_adjusted_r2_score(preds, dtrain):       #ADJUSTED R2
            labels = dtrain.get_label()
            n=dtrain.num_data()
            k=dtrain.num_feature()
            return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True
            
      # Select metric
      metric='lgb_r2_score'
      metric_feval=lgb_r2_score

      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':metric} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=metric_feval)
              #print(cv_results)
              return np.max(cv_results['metric-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (80, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.7, 1),
                  'max_depth': (7, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 25)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      return optimizer

bayesian_ouput=bayesian_opt_lgbm(X, y, init_iter=5, n_iters=500, random_state=77, seed = 101,num_iterations=300)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.5984   |  0.9757   |  0.6138   |  0.03793  |  8.115    |  11.31    |  0.07901  |  93.05    |
|  2        |  0.5634   |  0.8623   |  0.2922   |  0.02773  |  10.2     |  20.73    |  0.08383  |  103.5    |
|  3        |  0.5851   |  0.7888   |  0.3248   |  0.03557  |  10.38    |  10.86    |  0.07496  |  98.09    |
|  4        |  0.4617   |  0.7527   |  0.1395   |  0.01533  |  7.534    |  21.27    |  0.007313 |  97.28    |
|  5        |  0.5361   |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.54    |  0.08247  |  90.93    |
|  6        |  0.613    |  1.0      |  0.9      |  0.05     |  7.0      |  10.0     |  0.1      |  120.0    |
|  7        |  0.6253   |  1.0      |  0.9      |  0.05     |  15.0     |  25.0     |  0.001    |  120.0    |
|  8      

In [12]:
opt_parameters=bayesian_ouput.max['params']
opt_parameters

{'bagging_fraction': 0.9957675429106442,
 'feature_fraction': 0.8700275939500624,
 'learning_rate': 0.025998473221526312,
 'max_depth': 11.864045350159902,
 'min_child_weight': 24.803763598575678,
 'min_split_gain': 0.06485036212886282,
 'num_leaves': 91.1982883061857}

In [13]:
filename="opt_parameters_balanced_df_adjusted_{}_corr_r2_RISKY_EVENTS.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"))
a_file = open(filename, "wb")

pickle.dump(opt_parameters, a_file)

a_file.close()

In [14]:
# filename="opt_parameters_balanced_df_20210907_212730.pkl"
# a_file = open(filename,"rb")
# output = pickle.load(a_file)
# opt_parameters=output
# output

In [15]:
#optimizer.max['params']

In [16]:
#'bagging_fraction': 1.0, 'feature_fraction': 0.9, 'max_depth': 8.0, 'min_child_weight': 25.0, 'min_split_gain': 0.013771321931506838, 'num_leaves': 88.93816438820497}

In [17]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'regression_l2',
    'learning_rate': opt_parameters.get("learning_rate"),
    'feature_fraction': opt_parameters.get("feature_fraction"),
    'bagging_fraction': opt_parameters.get("bagging_fraction"),
    #'bagging_freq': 10,
    'verbose': -1,
    "max_depth": int(round(opt_parameters.get("max_depth"))),
    "num_leaves": int(round(opt_parameters.get("num_leaves"))),  
    #"max_bin": 512,
    'min_split_gain' : opt_parameters.get("min_split_gain"),
    "num_iterations": 300,
    "n_estimators": 500,
    'min_child_weight' : opt_parameters.get("min_child_weight")
}

In [18]:
#Construct a gradient boosting model.
#gbm = lgb.LGBMRegressor(**hyper_params)

In [19]:
lgbm_train = lgb.Dataset(X, label=y)
#lgbm_eval = lgb.Dataset(X_test, label=Y_test,reference=lgbm_train)

In [20]:
gbm = lgb.train(params=hyper_params,
                train_set=lgbm_train,
                #valid_sets=lgbm_eval,
                #verbose_eval=20,
                #eval_metric='lgb_r2_score',
                #early_stopping_rounds=100
                )

In [21]:
#Build a gradient boosting model from the training set (X, y)
""" gbm.fit(X, y,
        eval_set=[(X_test, Y_test)],
        eval_metric='l1',
        early_stopping_rounds=50) """


" gbm.fit(X, y,\n        eval_set=[(X_test, Y_test)],\n        eval_metric='l1',\n        early_stopping_rounds=50) "

In [22]:
Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [23]:
# print('The r2 of prediction is:', r2_score(y, Y_pred))
# print('The MSE of prediction is:', mean_squared_error(y, Y_pred, squared=True))
# print('The RMSE of prediction is:', mean_squared_error(y, Y_pred, squared=False))

In [24]:
print('The r2 of prediction is:', r2_score(Y_test, Y_pred))
print('The MSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=True))
print('The RMSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=False))
print('The MAE of prediction is:', mean_absolute_error(Y_test, Y_pred))

The r2 of prediction is: 0.8148868693024172
The MSE of prediction is: 22.570243369684327
The RMSE of prediction is: 4.750815021623588
The MAE of prediction is: 3.377156210629742


In [25]:
# df_train = lgb.Dataset(data=X_test, label=Y_test)
# def lgb_adjusted_r2_score(preds, dtrain):
#     labels = dtrain.get_label()
#     n=dtrain.num_data()
#     k=dtrain.num_feature()
#     return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True

In [26]:
#lgb_adjusted_r2_score(Y_pred, df_train)

### Validating model

In [27]:
aux_y=pd.DataFrame(Y_test)
aux_y.reset_index(inplace=True)
aux_y.drop(['index'], inplace=True, axis=1)
aux_y_pred=pd.DataFrame(Y_pred)
aux_y_pred.reset_index(inplace=True)
aux_y_pred.drop(['index'], inplace=True, axis=1)
frames=[aux_y,aux_y_pred]
result=pd.concat(frames,axis=1)
result.columns=["y_true","y_predicted"]
result["y_true_10"]=10**result.y_true
result["y_predicted_10"]=10**result.y_predicted
result

,y_true,y_predicted,y_true_10,y_predicted_10
0,-5.767512,-8.900231,1.708000e-06,1.258255e-09
1,-30.000000,-27.160980,1.000000e-30,6.902715e-28
2,-30.000000,-30.968100,1.000000e-30,1.076218e-31
3,-5.992252,-6.738717,1.018000e-06,1.825084e-07
4,-30.000000,-25.393743,1.000000e-30,4.038842e-26
5,-6.708187,-16.876747,1.958000e-07,1.328168e-17
6,-30.000000,-23.528650,1.000000e-30,2.960395e-24
7,-30.000000,-30.156739,1.000000e-30,6.970456e-31
8,-30.000000,-30.237428,1.000000e-30,5.788574e-31
9,-30.000000,-29.932838,1.000000e-30,1.167245e-30


In [28]:
result[result["y_true_10"]>0.00001]

,y_true,y_predicted,y_true_10,y_predicted_10
27,-4.387110,-5.118877,0.000041,7.605423e-06
31,-4.747390,-10.843474,0.000018,1.433922e-11
34,-3.346691,-11.470688,0.000450,3.383074e-12
43,-3.889410,-2.808278,0.000129,1.554970e-03
52,-4.557677,-4.725031,0.000028,1.883516e-05


In [29]:
result[result["y_true_10"]>0.0001]

,y_true,y_predicted,y_true_10,y_predicted_10
34,-3.346691,-11.470688,0.000450,3.383074e-12
43,-3.889410,-2.808278,0.000129,1.554970e-03
